# Type annotations in Python: The mypy static type-checker

The triggering situation for diving into type annotations and static type-checking was the following one:
 - an unclear/messy code,
 - inconvenient to read again after a break/difficult to remember subtle details,
 - unannotated/partially annotated code.
 
 The most common recommendation to tackle that kind of issues is to write DocStrings (in extenso documentation). But that 
 solution is dissatisfying:
 - DocStrings are tedious to write and not so flexible, especially for early code versions 
 which are are prone too (extensive) refactorization,
 - DocStrings are not very visual/readable, they have too much verbosity! That applies 
 in particular as soon as many variables/parameters are used.


## What are type annotations?
Python is a __dynamically-typed__ language. This means that the Python interpreter does type checking only as code runs,
and that the type of a variable is allowed to change over its lifetime.
Nevertheless, Python >= 3.0 allows type annotations (__standardized syntax__ for writing them described in __PEP 3107__):
- __Function annotations__ are written directly in the code, but they are __optional__.
They are __treated as comments__ at running time,
- Function annotations are __purely arbitrary__: the user associates Python expressions with parts of the code,
but __might not comply with the Python dynamically-set types__ at running time.
Pseudo-grammar for annotating a function and its parameters: _identifier [: expression] [= expression]_

Usually, information about annotations is accessible with the _annotations()_ method.


In [5]:
def foo(arg: object, kwarg: object = None) -> None:
    print('arg is a required parameter for foo() with value: ', arg)
    print('arg has type: ', type(arg))
    print('kwarg is an optional parameter for foo() with value: ', kwarg)
    print('kwarg has type: ', type(kwarg))
    
foo('bar')
print('\nAnnotations for the foo() function:\n', foo.__annotations__)

arg is a required parameter for foo() with value:  bar
arg has type:  <class 'str'>
kwarg is an optional parameter for foo() with value:  None
kwarg has type:  <class 'NoneType'>

Annotations for the foo() function:
 {'arg': <class 'object'>, 'kwarg': <class 'object'>, 'return': None}


As from Python 3.5, the _typing_ library provides extended types for annotating the Python code.
Annotations can be used for functions as well as for classes and objects. _typing_'s usage, when supported by 
the IDE, enables refined hinting functionality. As previously mentioned, the typing package does not introduce 
static typing of variables, but assigns arbitrary expressions. Those expressions re evaluated by the IDE 
in respect to the internal consistency of the package rules, which might not comply with the Python interpreter.

## What is static type-checking?
Static type-checking is a Python-independant eparate process for assessing consistency of annotations, 
based on the provided _built-ins_ or _typing_ types given.
One of the most mature project for applying static type-checking is the ___mypy___ project.
MyPy acts same way as linter, it simply checks your program for type errors by looking at type annotations.
So you pass your program to _mypy_ and you get error saying that you are passing invalid type to your function call.
This enables early debugging, among others. For further comparison based on users' experiences, please have a look at: 
https://www.reddit.com/r/Python/comments/6i5dwo/mypy_good_or_bad/

## How to get mypy?
Mypy can be installed via the regular _pip_ way. Note that a configuaration file is recommended 
(at your code's root for example) in order to easily customize your imports and checking options.
See teh official documentation: https://mypy.readthedocs.io/en/latest/

## Basic examples 

In [ ]:
# Example of configuration file: mypy.ini

In [ ]:
# commands for running mypy on a regular Python file:

# cd path/to/project
# mypy mytuples.py



In [1]:
%%mypy

def my_tuple0(a: int, b: int, c:int) -> tuple:
    return tuple((a, b, c))

t47_5 = my_tuple0(4, 7, -5)
print('A tuple of int: ', t47_5)
print('Type annotations of my_tuple function: \n', my_tuple0.__annotations__)


0

Equivalent to running _$ mypy mytuples.py_
Returns error code 0 i.e. no type errors checked in the piece of code.
NB: The code has not been read/interpreted/compiled with Python!


In [4]:
def my_tuple0(a: int, b: int, c:int) -> tuple:
    return tuple((a, b, c))

t47_5 = my_tuple0(4, 7, -5)
print('A tuple of int: ', t47_5)
print('Type annotations of my_tuple function: \n', my_tuple0.__annotations__)

A tuple of int:  (4, 7, -5)
Type annotations of my_tuple function: 
 {'a': <class 'int'>, 'b': <class 'int'>, 'c': <class 'int'>, 'return': <class 'tuple'>}


For running the piece of code with Python, remove the _%%mypy_ magic function. 
Equivalent to running _$ python mytuples.py_ instead of _$ mypy mytuples.py_


In [5]:
%%mypy

from typing import Tuple

def my_tuple1(a: int, b: int, c:int) -> Tuple[int]:
    return tuple((a, b, c))

t47_5 = my_tuple1(4, 7, -5)
print('A tuple of int: ', t47_5)
print('Type annotations of my_tuple function: \n', my_tuple1.__annotations__)



Type checking report:

<string>:5: error: Incompatible return value type (got "Tuple[int, ...]", expected "Tuple[int]")



1

Outputs error code 1 i.e. the mypy checking found potential type error(s).

The are several ways to suppress that error:
- comply with the suggestion returned by _mypy_,
- use a local silencer, 
- modify the local options in the _mypy.ini_ configuration file. 


In [9]:
from typing import Tuple

def my_tuple1(a: int, b: int, c:int) -> Tuple[int]:
    return tuple((a, b, c))

t47_5 = my_tuple1(4, 7, -5)
print('A tuple of int: ', t47_5)
print('Type annotations of my_tuple function: \n', my_tuple1.__annotations__)
print(type(t47_5))


A tuple of int:  (4, 7, -5)
Type annotations of my_tuple function: 
 {'a': <class 'int'>, 'b': <class 'int'>, 'c': <class 'int'>, 'return': typing.Tuple[int]}
<class 'tuple'>


Note here that even tagged with an error by _mypy_, the piece of code is Python-functional!
Python assigns dynamically the 'tuple' built-in type to the output.


In [7]:
%%mypy

from typing import Tuple, no_type_check

@no_type_check
def my_tuple1(a: int, b: int, c:int) -> Tuple[int]:
    return tuple((a, b, c))

t47_5 = my_tuple1(4, 7, -5)
print('A tuple of int: ', t47_5)
print('Type annotations of my_tuple function: \n', my_tuple1.__annotations__)


0

Here is how to locally silent the previous error by using a function decorator.


In [10]:
%%mypy

from typing import Tuple

def my_tuple2(a: int, b: int, c:int) -> Tuple[int, ...]:
    return tuple((a, b, c))

t47_5 = my_tuple2(4, 7, -5)
print('A tuple of int: ', t47_5)
print('Type annotations of my_tuple function: \n', my_tuple2.__annotations__)


0

Here is how to silent the previous error by following _mypy_ hints.


How to get an idea about the types that will be inferred by _mypy_ when writing a function or a class?

_mypy_ can understand two expressions for that purpose:
* _reveal_type(obj)_,
* _reveal_locals()_


In [11]:
%%mypy

from typing import Tuple

def my_tuple3(a, b, c):
    return tuple((a, b, c))

def my_tuple4(a:int, b:int, c:int) -> Tuple[int, ...]:
    return tuple((a, b, c))

t47_5 = my_tuple3(4, 7, -5)
t47_8 = my_tuple4(4, 7, -8)

# reveal_type(my_tuple3)  # underlined as error in Pycharm IDE!
# reveal_type(my_tuple4)  # unresolved reference reveal_type

reveal_type(t47_5)
reveal_type(t47_8)



Type checking report:

<string>:4: error: Function is missing a type annotation
<string>:16: error: Revealed type is 'Any'
<string>:17: error: Revealed type is 'builtins.tuple[builtins.int]'



1

In [8]:
# Run Python now:
from typing import Tuple

def my_tuple3(a, b, c):
    return tuple((a, b, c))

def my_tuple4(a:int, b:int, c:int) -> Tuple[int, ...]:
    return tuple((a, b, c))

t47_5 = my_tuple3(4, 7, -5)
t47_8 = my_tuple4(4, 7, -8)

# reveal_type(my_tuple3)  # underlined as error in Pycharm IDE!
# reveal_type(my_tuple4)  # unresolved reference reveal_type

reveal_type(t47_5)
reveal_type(t47_8)


NameError: name 'reveal_type' is not defined

_reveal_type_ and _reveal_locals_ are only understood by mypy and don’t exist in Python.
If you try to run your program, you’ll have to remove any reveal_type and reveal_locals calls
before you can run your code.
Both are always available and you don’t need to import them.


In [9]:
%%mypy

from typing import NewType, Tuple

# Here is an example on how to refine type annotation with typing
mytupletype = NewType('mytupletype', Tuple[float, float, bool])

def my_tuple4(a:int, b:int, c:int) -> mytupletype:
    return tuple((a, b, c))

t47_5 = my_tuple4(4, 7, -5)
t47_0 = my_tuple4(4, 7, 0)



Type checking report:

<string>:7: error: Incompatible return value type (got "Tuple[int, ...]", expected "mytupletype")



1

t47_5 outputs an error because -5 cannot be interpreted as a boolean, whereas
t47_0 do not return any error.


## Further examples 

In [ ]:
%%mypy

from typing import Any, TypeVar

def definition_domain0(x: Any) -> None: 
    if type(x) == str:
        print('x is defined by an alphabet')
    elif type(x) == bool:
        print('x tells the truth')
    elif type(x) == int:
        print('x is defined on |N')
    elif type(x) == float:
        print('x is defined on |R')
    else:
        print('What are you x?')
        
def definition_domain1(x: Any) -> Any: 
    if type(x) == str:
        print('x is defined by an alphabet')
    elif type(x) == bool:
        print('x tells the truth')
    elif type(x) == int:
        print('x is defined on |N')
    elif type(x) == float:
        print('x is defined on |R')
    else:
        print('What are you x?')
    return x

x_int = 8
x_bool = True
x_float = -3.07
x_str = 'å'
x_func = lambda  x: str(x)


y0 = definition_domain0(x_int)  # notified in Pycharm IDE

y11 = definition_domain1(x_int)
y12 = definition_domain1(x_str)
y13 = definition_domain1(x_func)

reveal_locals()


In [ ]:
%%mypy
from typing import Any, TypeVar, Union
       
Choosable = TypeVar('Choosable', str, bool, int, float)  # user-defined type
        
def definition_domain2(x: Choosable) -> Choosable: 
    if type(x) == str:
        print('x is defined by an alphabet')
    elif type(x) == bool:
        print('x tells the truth')
    elif type(x) == int:
        print('x is defined on |N')
    elif type(x) == float:
        print('x is defined on |R')
    else:
        pass
    return x

def definition_domain3(x: Union[str, bool, int, float]) -> Union[str, bool, int, float]: 
    if type(x) == str:
        print('x is defined by an alphabet')
    elif type(x) == bool:
        print('x tells the truth')
    elif type(x) == int:
        print('x is defined on |N')
    elif type(x) == float:
        print('x is defined on |R')
    else:
        pass
    return x

x_int = 8
x_bool = True
x_float = -3.07
x_str = 'å'
x_func = lambda  x: str(x)

y21 = definition_domain2(x_int)
y22 = definition_domain2(x_str)
y23 = definition_domain2(x_func)

y31 = definition_domain3(x_int)
y32 = definition_domain3(x_str)
y33 = definition_domain3(x_func)

reveal_locals()


In [ ]:
%%mypy

class Shadok(object):
    def __init__(self, name: str) -> None:
        self.name = name

    def assign_task(self, state: str) -> None:
        self.__setattr__('state', state)
        print('Shadok {} is now {}'.format(repr(self.name), self.__getattribute__('state')))
        
s1 = Shadok('shadok1')
print(s1.__dict__)
print(s1.__annotations__) 
print('\n')

s1.assign_task('pumping')
print(s1.__dict__)
print(s1.__annotations__)
print(Shadok.__init__.__annotations__)
print('\n')

reveal_type(s1)
reveal_type(s1.__class__)
reveal_type(s1.name)
reveal_type(s1.state)  # 'Shadok' has no attribute state


MyPy is able to infer the types of class attributes, but the result is not so relevant


In [ ]:
class Shadok(object):
    def __init__(self, name: str) -> None:
        self.name = name

    def assign_task(self, state: str) -> None:
        self.__setattr__('state', state)
        print('Shadok {} is now {}'.format(repr(self.name), self.__getattribute__('state')))
        
s1 = Shadok('shadok1')
print('s1.__dict__: ', s1.__dict__)
# print(s1.__annotations__)  # s1 has no annotations! s1 is an instance
# print(Shadock.__annotations__)  # Shadock has no annotations! Shadok is a class
print('Shadok.__init__.__annotations__: ', Shadok.__init__.__annotations__)
print('Shadok.assign_task.__annotations__: ', Shadok.assign_task.__annotations__)
print('\n')

s1.assign_task('pumping')
print('s1.__dict__: ', s1.__dict__) # 'Shadok' has an attribute state!
# print('s1.name.__annotations__: ', s1.name.__annotations__) # AttributeError: 'str' object has no annotations

Only the functions are here annotated, not the instances


## Create your own customized types
For instance for my research projects, using genetic data encoded as genotypes:

Genotypes formats, with R = reference allele = 0 and A = alternate allele = 1:

| Genotype | GenotypeTrue (phased or unphased) | GenotypeLikelihoods (likelihoods of (RR, {RA,AR}, AA) |
| :-: | :------------: | :-: |
RR | 0\|0 -> (0, 0, 1) or 0/0 -> (0, 0, 0) | (1, 0, 0) |
RA | 0\|1 -> (0, 1, 1) or 0/1 -> (0, 1, 0) | (0, 1, 0)
AR | 1\|0 -> (1, 0, 1) or 1/0 -> (1, 0, 0) | (0, 1, 0)
AA | 1\|1 -> (1, 1, 1) or 1/1 -> (1, 1, 0) | (0, 0, 1)


In [ ]:
%%mypy

from typing import NewType, List

GL = NewType('GenotypeLikelihoods', Tuple[float, float, float]) 
GT = NewType('GenotypeTrue', Tuple[int, int, bool])


In [7]:
%%mypy

from typing import NewType, List

GLtype = NewType('GLtype', Tuple[float, float, float])
GTtype = NewType('GTtype', Tuple[int, int, bool])


Type checking report:

<string>:4: error: Argument 2 to NewType(...) must be subclassable (got "Any")
<string>:4: error: Name 'Tuple' is not defined
<string>:5: error: Argument 2 to NewType(...) must be subclassable (got "Any")
<string>:5: error: Name 'Tuple' is not defined



1

In [11]:
# %%mypy

# Customize the str type for typing files paths:
from pathlib import Path
from os import PathLike
from typing import NewType, Union

FilePath = NewType('FilePath', Union[Path, PathLike])


/home/camille/PycharmProjects
None


References/further information:

- https://docs.python.org/3/reference/compound_stmts.html#function
- https://docs.python.org/3/glossary.html#term-function-annotation
- https://www.python.org/dev/peps/pep-3107/
- https://docs.python.org/3/library/typing.html
- https://mypy.readthedocs.io/en/latest/index.html
- https://realpython.com/python-type-checking
- https://switowski.com/python/ipython/2019/02/15/creating-magic-functions-part3.html



